In [1]:
import pandas as pd
import datetime
import logging
from sklearn.model_selection import KFold
import argparse
import json
import numpy as np

import sys
sys.path.append('..')
from utils import load_datasets, load_target
from logs.logger import log_best
from models.lgbm import train_and_predict
import os
os.chdir('../')

%matplotlib inline

In [2]:
os.getcwd()

'C:\\Users\\takuy\\work\\work_tokumoto\\splatoon_competition'

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', default='./configs/default_v04.json')
options = parser.parse_args(args=[])
config = json.load(open(options.config))

feats = config['features']
logging.debug(feats)

target_name = config['target_name']

In [4]:
X_train_all, X_test = load_datasets(feats, target_name)
y_train_all = load_target(target_name)

In [5]:
X_train_all.head()

,A1-level,A2-level,A3-level,A4-level,B1-level,B2-level,B3-level,B4-level,enc_period,enc_game-ver,...,poisonmist_B_count,torpedo_A_count,torpedo_B_count,jumpbeacon_A_count,jumpbeacon_B_count,tansanbomb_A_count,tansanbomb_B_count,trap_A_count,trap_B_count,size
0,139,118.0,13.0,10.0,28,26.0,68.0,31.0,0.515152,0.525161,...,0,0,0,0,0,0,0,1,0,2855.0
1,198,77.0,198.0,123.0,83,118.0,168.0,151.0,0.466667,0.524499,...,0,0,1,0,1,0,0,1,0,2391.0
2,114,68.0,225.0,107.0,50,163.0,160.0,126.0,0.600000,0.525747,...,0,0,0,0,0,0,0,0,0,2426.0
3,336,131.0,189.0,41.0,273,189.0,194.0,391.0,0.571429,0.523781,...,0,0,0,0,0,0,0,0,1,2237.4
4,299,97.0,96.0,136.0,101,45.0,246.0,160.0,0.580000,0.524329,...,1,0,0,0,0,0,0,0,0,2390.0


In [14]:
X_train_all[X_train_all["A_charger_count"] == 2]["A_charger_count"]

8        2
12       2
35       2
72       2
97       2
102      2
109      2
145      2
153      2
158      2
285      2
307      2
352      2
383      2
411      2
480      2
515      2
539      2
571      2
616      2
619      2
623      2
631      2
677      2
698      2
778      2
793      2
907      2
1009     2
1012     2
        ..
65291    2
65316    2
65370    2
65379    2
65391    2
65411    2
65428    2
65445    2
65570    2
65576    2
65589    2
65608    2
65622    2
65632    2
65700    2
65745    2
65752    2
65778    2
65810    2
65813    2
65833    2
65862    2
65897    2
65974    2
65988    2
66001    2
66030    2
66103    2
66108    2
66117    2
Name: A_charger_count, Length: 1991, dtype: int64

In [6]:
# 訓練データとテストデータの列を確認
print(X_train_all.columns)
print(X_test.columns)

Index(['A1-level', 'A2-level', 'A3-level', 'A4-level', 'B1-level', 'B2-level',
       'B3-level', 'B4-level', 'enc_period', 'enc_game-ver',
       ...
       'poisonmist_B_count', 'torpedo_A_count', 'torpedo_B_count',
       'jumpbeacon_A_count', 'jumpbeacon_B_count', 'tansanbomb_A_count',
       'tansanbomb_B_count', 'trap_A_count', 'trap_B_count', 'size'],
      dtype='object', length=125)
Index(['A1-level', 'A2-level', 'A3-level', 'A4-level', 'B1-level', 'B2-level',
       'B3-level', 'B4-level', 'enc_period', 'enc_game-ver',
       ...
       'poisonmist_B_count', 'torpedo_A_count', 'torpedo_B_count',
       'jumpbeacon_A_count', 'jumpbeacon_B_count', 'tansanbomb_A_count',
       'tansanbomb_B_count', 'trap_A_count', 'trap_B_count', 'size'],
      dtype='object', length=125)


In [7]:
# 訓練データとテストデータの列を確認
print(X_train_all.shape)
print(X_test.shape)

(66125, 125)
(28340, 125)


In [12]:
#　訓練データに欠損がないことの確認
X_train_all.isnull().sum().sum()

0

In [9]:
#　テストデータに欠損がないことの確認
X_test.isnull().sum().sum()

0

In [10]:
print("-------------------train-------------------")
print(X_train_all.isnull().sum())
print("-------------------test-------------------")
print(X_test.isnull().sum())

-------------------train-------------------
A1-level          0
A2-level          0
A3-level          0
A4-level          0
B1-level          0
B2-level          0
B3-level          0
B4-level          0
enc_period        0
enc_game-ver      0
enc_lobby-mode    0
enc_lobby         0
enc_mode          0
enc_stage         0
enc_A1-weapon     0
enc_A1-rank       0
enc_A2-weapon     0
enc_A2-rank       0
enc_A3-weapon     0
enc_A3-rank       0
enc_A4-weapon     0
enc_A4-rank       0
enc_B1-weapon     0
enc_B1-rank       0
enc_B2-weapon     0
enc_B2-rank       0
enc_B3-weapon     0
enc_B3-rank       0
enc_B4-weapon     0
enc_B4-rank       0
dtype: int64
-------------------test-------------------
A1-level          0
A2-level          0
A3-level          0
A4-level          0
B1-level          0
B2-level          0
B3-level          0
B4-level          0
enc_period        0
enc_game-ver      0
enc_lobby-mode    0
enc_lobby         0
enc_mode          0
enc_stage         0
enc_A1-weapon     0


In [14]:
display(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28340 entries, 0 to 28339
Data columns (total 30 columns):
A1-level          28340 non-null int64
A2-level          28340 non-null float64
A3-level          28340 non-null float64
A4-level          28340 non-null float64
B1-level          28340 non-null int64
B2-level          28340 non-null float64
B3-level          28340 non-null float64
B4-level          28340 non-null float64
enc_period        28340 non-null float64
enc_game-ver      28340 non-null float64
enc_lobby-mode    28340 non-null float64
enc_lobby         28340 non-null float64
enc_mode          28340 non-null float64
enc_stage         28340 non-null float64
enc_A1-weapon     28340 non-null float64
enc_A1-rank       28340 non-null float64
enc_A2-weapon     28340 non-null float64
enc_A2-rank       28340 non-null float64
enc_A3-weapon     28340 non-null float64
enc_A3-rank       28340 non-null float64
enc_A4-weapon     28340 non-null float64
enc_A4-rank       28340 non-null f

None